In [15]:
using LinearAlgebra

In [10]:
function classic_(A, B, C)
    len = size(A,1)
    for i in 1:len
       for j in 1:len
           for k in 1:len
                C[i, j] += (A[i, k] * B[k, j])
            end
        end
    end
    return C
end

classic_ (generic function with 1 method)

In [78]:
function multiply_rec(A,B)
    C = zeros(size(A, 1), size(A,1))
    if size(A,1) == 2
       classic_(A, B, C)
        return C
    end
    len = (size(A,1)÷2)
    idx = [ [1:len,1:len],
            [1:len,len+1:len*2],
            [len+1:len*2,1:len],
            [len+1:len*2,len+1:len*2]
          ]

    C[idx[1][1], idx[1][2]] .= multiply_rec(A[idx[1][1], idx[1][2]], 
                 B[idx[1][1], idx[1][2]],
                 C[idx[1][1], idx[1][2]])

    C[idx[1][1], idx[1][2]] .= multiply_rec(A[idx[2][1], idx[2][2]], 
                 B[idx[3][1], idx[3][2]],
                 C[idx[1][1], idx[1][2]])

    C[idx[2][1], idx[2][2]] .= multiply_rec(A[idx[1][1], idx[1][2]], 
                 B[idx[2][1], idx[2][2]],
                C[idx[2][1], idx[2][2]])
    C[idx[2][1], idx[2][2]] .= multiply_rec(A[idx[2][1], idx[2][2]], 
                 B[idx[4][1], idx[4][2]],
                 C[idx[2][1], idx[2][2]])
    
    C[idx[3][1], idx[3][2]] .= multiply_rec(A[idx[3][1], idx[3][2]], 
                 B[idx[1][1], idx[1][2]],
                 C[idx[3][1], idx[3][2]])
    C[idx[3][1], idx[3][2]] .= multiply_rec(A[idx[4][1], idx[4][2]], 
                 B[idx[3][1], idx[3][2]],
                 C[idx[3][1], idx[3][2]])
    
    C[idx[4][1], idx[4][2]] .= multiply_rec(A[idx[3][1], idx[3][2]], 
                 B[idx[2][1], idx[2][2]],
                 C[idx[4][1], idx[4][2]])
    C[idx[4][1], idx[4][2]] .= multiply_rec(A[idx[4][1], idx[4][2]], 
                 B[idx[4][1], idx[4][2]],
                 C[idx[4][1], idx[4][2]])
    return C
end

multiply_rec (generic function with 2 methods)

In [123]:
function inv2x2(A)
    detA = A[1, 1]*A[2, 2] - A[1, 2]*A[2, 1]
    invA = [
        A[2, 2] -A[1, 2]
        -A[2, 1] A[1, 1]
    ]
    return (1/detA)*invA
end

function inv(A)
    len    = (size(A,1)÷2)
    invA11 = 1*zeros(len, len)
    B      = 1*zeros(size(A,1), size(A,1))
    S22    = 1*zeros(len, len)
    invS22 = 1*zeros(len, len)
    
    if size(A, 1) == 2
        invA = inv2x2(A)
        return invA
    end
    
    idx = [ [1:len,1:len],
            [1:len,len+1:len*2],
            [len+1:len*2,1:len],
            [len+1:len*2,len+1:len*2]
          ]

    invA11= inv(A[idx[1][1], idx[1][2]])
    
    S22 = A[idx[4][1], idx[4][2]] - 
            (multiply_rec(
                    multiply_rec(A[idx[3][1], idx[3][2]], invA11), A[idx[2][1], idx[2][2]]))
    invS22 = inv(S22)
    Id = 1.0*Matrix(I, len, len)
    
    B[idx[1][1], idx[1][2]] = multiply_rec(invA11, 
    (Id + multiply_rec(multiply_rec(multiply_rec(A[idx[2][1],idx[2][2]], invS22), A[idx[3][1], idx[3][2]]), invA11)))
    B[idx[2][1], idx[2][2]] = multiply_rec(multiply_rec(-invA11, A[idx[2][1], idx[2][2]]), invS22)
    B[idx[3][1], idx[3][2]] = multiply_rec(multiply_rec(-invS22, A[idx[3][1], idx[3][2]]), invA11)
    B[idx[4][1], idx[4][2]] = invS22
    
    return B
end

inv (generic function with 2 methods)

In [124]:
b = [
    3.0 4.0 2.0 4.0
    1.0 4.0 3.0 4.0
    3.0 1.0 3.0 2.0
    3.0 4.0 3.0 4.0
]

invB = inv(b)

4×4 Matrix{Float64}:
  0.0   -0.5     0.0   0.5
 -1.5   -0.75   -1.0   2.75
 -1.0    0.0    -0.0   1.0
  2.25   1.125   1.0  -3.625

In [125]:
b = [
    3.0 4.0 
    1.0 4.0 
]
println(b[1:2, 1:2])
invB = inv(b)

[3.0 4.0; 1.0 4.0]


2×2 Matrix{Float64}:
  0.5    -0.5
 -0.125   0.375

In [81]:
a = [
    1 2 1 2
    1 2 1 2
    1 2 1 2
    1 2 1 2
]

b = [
    3 4 3 4
    3 4 3 4
    3 4 3 4
    3 4 3 4
]

c = [
    0 0 0 0
    0 0 0 0
    0 0 0 0
    0 0 0 0
]

c = multiply_rec(a, b)
print(c)

[18.0 24.0 18.0 24.0; 18.0 24.0 18.0 24.0; 18.0 24.0 18.0 24.0; 18.0 24.0 18.0 24.0]